In [19]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sys

#instant execution, useful for dev
tf.enable_eager_execution()

print(tf.__version__)
print(tf.keras.__version__)
print(pd.__version__)
print(np.__version__)
print (sys.version_info)

1.13.1
2.2.4-tf
0.22.0
1.14.6
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)


In [20]:
#load data
from google.colab import drive
drive.mount('/content/drive/')

dftrain=pd.read_csv('/content/drive/My Drive/train.csv')
dfpredict=pd.read_csv('/content/drive/My Drive/test.csv')

print(dftrain.head())
print(dftrain.shape[0])
print(dfpredict.head())
print(dfpredict.shape[0])

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
   Store  Dept        Date  Seasonality  Year      Size  Type_A  Type_B  \
0      1     1  2010-02-05     0.078431   0.0  0.630267       1       0   
1      1     1  2010-02-12     0.098039   0.0  0.630267       1       0   
2      1     1  2010-02-19     0.117647   0.0  0.630267       1       0   
3      1     1  2010-02-26     0.137255   0.0  0.630267       1       0   
4      1     1  2010-03-05     0.156863   0.0  0.630267       1       0   

   Type_C  Temperature  Fuel_Price  IsHoliday_False  IsHoliday_True  Store2  \
0       0     0.454046    0.050100                1               0     0.0   
1       0     0.419260    0.038076                0               1     0.0   
2       0     0.432259    0.021042                1               0     0.0   
3       0     0.493592    0.044589                1               0     0.0   
4       0     0.492402 

In [21]:
#I don't need the Date,Store and Dept columns for NN (but used for submission)
dftrain.drop("Date", axis=1, inplace=True) 
dftrain.drop("Store", axis=1, inplace=True) 
dftrain.drop("Dept", axis=1, inplace=True) 
dfpredict.drop("Date", axis=1, inplace=True) 
dfpredict.drop("Store", axis=1, inplace=True) 
dfpredict.drop("Dept", axis=1, inplace=True) 

print(dftrain.shape[0])
print(dfpredict.shape[0])

421570
115064


In [0]:
#force all types to float32
dftrain = dftrain.astype(np.float32)
dfpredict = dfpredict.astype(np.float32)

#split train data in features and labels
y = dftrain.Weekly_Sales
x = dftrain.drop('Weekly_Sales', axis=1)

#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [0]:
#convert the dataframes to numpy arrays
#y_train = y_train.to_numpy() requires 0.24
y_train = y_train.values
x_train = x_train.values
y_test = y_test.values
x_test = x_test.values
x_predict = dfpredict.values

In [0]:
#here is the NN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=7, activation='relu', input_dim=12))
model.add(Dense(units=1, activation='linear'))
model.compile(loss='mean_squared_error',
              optimizer='Adam',
              metrics=['accuracy'])

In [25]:
#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=250)

Train on 337256 samples, validate on 84314 samples
Epoch 1/250
337256/337256 [==============================] - 13s 38us/sample - loss: 741737350.0593 - acc: 4.0622e-04 - val_loss: 683305962.9480 - val_acc: 3.7953e-04
Epoch 2/250
337256/337256 [==============================] - 12s 36us/sample - loss: 620834938.7917 - acc: 4.0325e-04 - val_loss: 553730245.7661 - val_acc: 3.7953e-04
Epoch 3/250
337256/337256 [==============================] - 12s 36us/sample - loss: 525451707.8369 - acc: 4.0325e-04 - val_loss: 503896028.1766 - val_acc: 3.7953e-04
Epoch 4/250
337256/337256 [==============================] - 12s 36us/sample - loss: 504696596.7688 - acc: 4.0325e-04 - val_loss: 497882571.4786 - val_acc: 3.7953e-04
Epoch 5/250
337256/337256 [==============================] - 12s 37us/sample - loss: 500201959.2749 - acc: 4.0325e-04 - val_loss: 494182151.2385 - val_acc: 3.7953e-04
Epoch 6/250
337256/337256 [==============================] - 12s 37us/sample - loss: 496831049.4914 - acc: 4.0325e

In [26]:
# Predict!!
y_predict = model.predict(x_predict)

#for i in range(len(x_predict)):
for i in range(10):
	print("x=%s, y=%s" % (x_predict[i], y_predict[i]))

x=[0.84313726 0.6666667  0.6302673  1.         0.         0.
 0.5731417  0.45791584 1.         0.         0.         0.        ], y=[14934.559]
x=[0.8627451  0.6666667  0.6302673  1.         0.         0.
 0.6273343  0.42184368 1.         0.         0.         0.        ], y=[14821.707]
x=[0.88235295 0.6666667  0.6302673  1.         0.         0.
 0.5511717  0.39078155 1.         0.         0.         0.        ], y=[14862.062]
x=[0.9019608  0.6666667  0.6302673  1.         0.         0.
 0.581472   0.37024048 0.         1.         0.         0.        ], y=[17557.143]
x=[0.92156863 0.6666667  0.6302673  1.         0.         0.
 0.5458623  0.36823648 1.         0.         0.         0.        ], y=[14885.962]
x=[0.9411765  0.6666667  0.6302673  1.         0.         0.
 0.65369827 0.36372745 1.         0.         0.         0.        ], y=[14796.284]
x=[0.9607843 0.6666667 0.6302673 1.        0.        0.        0.5142805
 0.3486974 1.        0.        0.        0.       ], y=[14945.8

In [27]:
#save predictions to csv
from google.colab import drive
drive.mount('/content/drive/')
np.savetxt("/content/drive/My Drive/prediction.csv", y_predict, fmt="%10.5f", delimiter=",")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
